In [ ]:
!pip install accelerate optimum

In [ ]:
!pip install auto-gptq

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
# To use a different branch, change revision
# For example: revision="main"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="cuda",
                                             torch_dtype=torch.float16,
                                             trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)


In [ ]:
import json

# Load JSON file
with open('/content/drive/MyDrive/Project/dataset/coin_flip.json', 'r') as file:
    data = json.load(file)

# Initialize an empty list to store results
results = []

In [ ]:
count = 0
for item in data['data']:
    # Get prompt details
    prompt = item["question"]
    count= count+1
    print(count)
    # prompt_template=f'''[INST] <<SYS>>
    # You are an online math tutor bot. Follow the given instructions: Given the below-structured query,
    # your job is to understand the word problem mentioned as "question" and explain the solution in numbered
    # steps to get the correct "answer". Begin your response with "let's think step by step" and append Answer: ["answer" from the
    # below-structured query] at the end of your reasoning. Strictly limit the reasoning to 20-30 words. Do not use emojis.{prompt}[/INST]
    # '''
    prompt_template=f'''[INST] <<SYS>>Lets think step by step. {prompt}[/INST]
    '''
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.1,
        top_p=0.95,
        top_k=1,
        repetition_penalty=1.1
    )
    response = pipe(prompt_template)[0]['generated_text']
    response = response.split('[/INST]')[1].strip()

    results.append({'prompt': item['question'], 'reasoning': response, 'answer': item['answer']})
    if count==300:
      break


In [ ]:
# print(results)

In [ ]:
import json

In [ ]:
with open('/content/drive/MyDrive/Project/Training_datasets/coin-flip-training.json', 'w') as file:
     json.dump(results, file, indent=4)